In [6]:
import tensorflow as tf

from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.layers import Conv2D,BatchNormalization,Activation,Add,Flatten,Dense,Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

from kerastuner import HyperModel
from kerastuner.tuners import RandomSearch



In [4]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.8 MB/s eta 0:00:00


In [7]:
# veri setini yükle
(train_images,train_labels),(test_images,test_labels) = fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [8]:
# veri setinin reshape yapalım (28x28) ->(28x28x1)
train_images = train_images.reshape(-1,28,28,1).astype("float32")/255.0
test_images = test_images.reshape(-1,28,28,1).astype("float32")/255.0

In [9]:
#etiketleri one-hot encoding ile hazırlayalım
train_labels = to_categorical(train_labels,10)
test_labels = to_categorical(test_labels,10)

In [15]:
#residual block

def residual_block(x,filters,kernel_size=3,strides=1):

  shortcut = x

  #1.conv katmanı
  x = Conv2D(filters,kernel_size=kernel_size,strides=strides,padding="same")(x)
  x = BatchNormalization()(x)# nırmalizasyon işlemi
  x = Activation("relu")(x)# aktivasyon fonksiyonu

  #2.conv katmanı
  x = Conv2D(filters,kernel_size=kernel_size,strides=strides,padding="same")(x)
  x = BatchNormalization()(x)

  # eğer girişten gelen verinin boyutu filtre sayısına eşit değilse
  if shortcut.shape[-1] != filters:
    shortcut = Conv2D(filters,kernel_size=1,strides=strides,padding="same")(shortcut)
    shortcut = BatchNormalization()(shortcut)

  #residual bağlantısı: giriş verisi işe çıkış verisini toplayalım
  x = Add()([x,shortcut])
  x = Activation("relu")(x)

  return x

In [11]:
# create and compile resnet

class ResNetModel(HyperModel):

  def build(self,hp): #hp = hyper parameter tuning için kullanılacak parametre

    inputs = Input(shape=(28,28,1))

    # 1.conv layer
    x = Conv2D(filters=hp.Int("inital_filters",min_value=32,max_value=128,step=32),
               kernel_size=3,strides=1,padding="same",activation="relu")(inputs)
    x = BatchNormalization()(x)

    #residual block ekleyelim
    for i in range(hp.Int("num_blocks",min_value=1,max_value=3,step=1)):
      x = residual_block(x,hp.Int("res_filters",min_value=32,max_value=128,step00=32))

    #sınıflandırma katmanı
    x=Flatten()(x)
    x = Dense(10,activation="relu")(x)
    outputs = Dense(10,activation="softmax")(x)

    model = Model(inputs=inputs,outputs=outputs)

    # model compile
    model.compile(optimizer = Adam(hp.Float("learning_rate",min_value=1e-4,max_value=1e-2,sampling="log")),
                  loss = "categorical_crossentropy",
                  metrics = ["accuracy"])

    return model

In [13]:
# create and compile resnet

class ResNetModel(HyperModel):

  def build(self,hp): #hp = hyper parameter tuning için kullanılacak parametre

    inputs = Input(shape=(28,28,1))

    # 1.conv layer
    x = Conv2D(filters=hp.Int("inital_filters",min_value=32,max_value=128,step=32),
               kernel_size=3,strides=1,padding="same",activation="relu")(inputs)
    x = BatchNormalization()(x)

    #residual block ekleyelim
    for i in range(hp.Int("num_blocks",min_value=1,max_value=3,step=1)):
      # Replace 'step00' with 'step'
      x = residual_block(x,hp.Int("res_filters",min_value=32,max_value=128,step=32))

    #sınıflandırma katmanı
    x=Flatten()(x)
    x = Dense(10,activation="relu")(x)
    outputs = Dense(10,activation="softmax")(x)

    model = Model(inputs=inputs,outputs=outputs)

    # model compile
    model.compile(optimizer = Adam(hp.Float("learning_rate",min_value=1e-4,max_value=1e-2,sampling="log")),
                  loss = "categorical_crossentropy",
                  metrics = ["accuracy"])

    return model

In [ ]:
tuner = RandomSearch(
    ResNetModel(),
    objective="val_accuracy", # tuning referans değeri
    max_trials=2,#toplam deneme sayısı
    executions_per_trial = 1, # her denemede kaç kere eğitim yapılacağı
    project_name="resnet_model_tuning"
)

#hiperparametre optimizasyonu and training
tuner.search(train_images,train_labels,epochs=1,validation_data=(test_images,test_labels))

In [ ]:
# en iyi modeli alalım
best_model = tuner.get_best_models(num_models=1)[0]

# test seti ile em iyi modeli test edelim
test_loss,test_acc = best_model.evaluate(test_images,test_labels)
print(f"Test accuracy: {test_acc}, Test loss: {test_loss}")